# Loan 파일로 복습

### 0. 라이브러리

In [82]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer


In [36]:
loan=pd.read_csv('loan.csv')

In [37]:
loan

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,Female,No,0,Graduate,No,2900,0.0,71.0,360.0,1.0,Rural,Y
610,LP002979,Male,Yes,3+,Graduate,No,4106,0.0,40.0,180.0,1.0,Rural,Y
611,LP002983,Male,Yes,1,Graduate,No,8072,240.0,253.0,360.0,1.0,Urban,Y
612,LP002984,Male,Yes,2,Graduate,No,7583,0.0,187.0,360.0,1.0,Urban,Y


In [38]:
loan.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

### 1. 전처리

#### 1> null값 채우기

In [39]:
loan['Gender'].value_counts(dropna=False) #null값 까지 표시하기 위해 dropna=False

Male      489
Female    112
NaN        13
Name: Gender, dtype: int64

In [40]:
loan['Gender'].mode() #최빈값!!

0    Male
Name: Gender, dtype: object

In [41]:
male=loan['Gender'].value_counts(dropna=False)[0]
female=loan['Gender'].value_counts(dropna=False)[1]
na=loan['Gender'].value_counts(dropna=False)[2]

In [42]:
print('Male의 비율 : %.2f'%(male / len(loan) *100))
print('Female의 비율 : %.2f'%(female / len(loan) *100))
print('Null의 비율 : %.2f'%(na / len(loan) *100))

Male의 비율 : 79.64
Female의 비율 : 18.24
Null의 비율 : 2.12


In [43]:
loan['Gender'].fillna(loan['Gender'].mode()[0], inplace=True) #Gender 최빈값으로 넣음

In [44]:
loan['Gender'].isnull().sum()

0

In [45]:
loan['Married'].value_counts(dropna=False)

Yes    398
No     213
NaN      3
Name: Married, dtype: int64

In [46]:
loan['Married'].fillna(loan['Married'].mode()[0], inplace=True)

In [47]:
loan['Married'].isnull().sum()

0

In [48]:
null_col = ['Gender','Married','Dependents','Self_Employed'] #4가지 항목은 최빈값으로 결측치 처리!

In [49]:
for col in null_col :
    #print(loan[col].value_counts(dropna=False))
    loan[col].fillna(loan[col].mode()[0], inplace=True)

In [50]:
loan.isnull().sum()

Loan_ID               0
Gender                0
Married               0
Dependents            0
Education             0
Self_Employed         0
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [51]:
loan['LoanAmount'].describe()

count    592.000000
mean     146.412162
std       85.587325
min        9.000000
25%      100.000000
50%      128.000000
75%      168.000000
max      700.000000
Name: LoanAmount, dtype: float64

In [52]:
loan['LoanAmount'].median()

128.0

In [53]:
loan['Self_Employed'].value_counts()

No     532
Yes     82
Name: Self_Employed, dtype: int64

In [54]:
loan.groupby('Self_Employed')['LoanAmount'].median()

Self_Employed
No     125.0
Yes    150.0
Name: LoanAmount, dtype: float64

In [55]:
loan[loan['LoanAmount'].isnull()]['Self_Employed'].value_counts()

No     19
Yes     3
Name: Self_Employed, dtype: int64

In [56]:
loan['LoanAmount'].fillna(loan['LoanAmount'].median(), inplace=True) # LoanAmount의 결측치 처리 중위수로!

In [57]:
loan['LoanAmount'].isnull().sum()

0

In [58]:
loan['Loan_Amount_Term'].value_counts()

360.0    512
180.0     44
480.0     15
300.0     13
240.0      4
84.0       4
120.0      3
60.0       2
36.0       2
12.0       1
Name: Loan_Amount_Term, dtype: int64

In [59]:
loan.groupby('Loan_Amount_Term')['LoanAmount'].median()

Loan_Amount_Term
12.0     111.0
36.0     117.5
60.0     140.0
84.0     126.0
120.0     25.0
180.0    127.0
240.0    114.0
300.0    124.0
360.0    128.0
480.0    113.0
Name: LoanAmount, dtype: float64

In [60]:
loan['Loan_Amount_Term'].fillna(loan['Loan_Amount_Term'].mode()[0], inplace=True) # LoanAmountterm의 결측치 처리 최빈값로!

In [61]:
loan['Credit_History'].value_counts(dropna=False)

1.0    475
0.0     89
NaN     50
Name: Credit_History, dtype: int64

In [62]:
loan.groupby('Credit_History')['LoanAmount'].median()

Credit_History
0.0    127.0
1.0    128.0
Name: LoanAmount, dtype: float64

In [63]:
loan['Credit_History'].fillna(loan['Credit_History'].mode()[0], inplace=True) # LoanAmountterm의 결측치 처리 최빈값로!

In [64]:
loan.isnull().sum()

Loan_ID              0
Gender               0
Married              0
Dependents           0
Education            0
Self_Employed        0
ApplicantIncome      0
CoapplicantIncome    0
LoanAmount           0
Loan_Amount_Term     0
Credit_History       0
Property_Area        0
Loan_Status          0
dtype: int64

In [87]:
null_dict ={}

null_col = ['Gender','Married','Dependents','Self_Employed','Loan_Amount_Term','Credit_History']

for col in null_col :
    null_dict[col] =loan[col].mode()[0]
    
null_dict['LoanAmount']= loan['LoanAmount'].median()
# null값을 채울 값을 null_dict에 저장

In [88]:
null_dict

{'Gender': 1,
 'Married': 1,
 'Dependents': 0,
 'Self_Employed': 0,
 'Loan_Amount_Term': 360.0,
 'Credit_History': 1.0,
 'LoanAmount': 128.0}

In [90]:
null_col = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']
for col in null_col :
    loan[col].fillna(null_dict[col], inplace=True) #null값을 null_dict에 해당값으로 채워줌

In [91]:
null_dict={'Gender': 1, 'Married': 1, 'Dependents': 0, 'Self_Employed': 0, 'Loan_Amount_Term': 360.0, 'Credit_History': 1.0, 'LoanAmount': 128.0}
null_col = ['Gender', 'Married', 'Dependents', 'Self_Employed', 'LoanAmount', 'Loan_Amount_Term', 'Credit_History']

def fill_null(data) :
    for col in null_col :
        loan[col].fillna(null_dict[col], inplace=True)
    return data
    

#### 2> object변수 LabelEncoder 적용

In [65]:
obj_col = ['Gender','Married','Dependents','Education','Self_Employed','Property_Area']

In [66]:
for col in obj_col :
    label = LabelEncoder()
    loan[col] = label.fit_transform(loan[col]) 

In [67]:
loan

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1,0,0,0,0,5849,0.0,128.0,360.0,1.0,2,Y
1,LP001003,1,1,1,0,0,4583,1508.0,128.0,360.0,1.0,0,N
2,LP001005,1,1,0,0,1,3000,0.0,66.0,360.0,1.0,2,Y
3,LP001006,1,1,0,1,0,2583,2358.0,120.0,360.0,1.0,2,Y
4,LP001008,1,0,0,0,0,6000,0.0,141.0,360.0,1.0,2,Y
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,LP002978,0,0,0,0,0,2900,0.0,71.0,360.0,1.0,0,Y
610,LP002979,1,1,3,0,0,4106,0.0,40.0,180.0,1.0,0,Y
611,LP002983,1,1,1,0,0,8072,240.0,253.0,360.0,1.0,2,Y
612,LP002984,1,1,2,0,0,7583,0.0,187.0,360.0,1.0,2,Y


In [ ]:
for col in obj_col :
    label = LabelEncoder()
    loan[col] = label.fit_transform(loan[col])

### 2. 모델링

In [68]:
X = loan.drop(['Loan_ID','Loan_Status'], axis=1)
Y = loan['Loan_Status']

In [69]:
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, stratify=Y, random_state=1)

In [70]:
model = SVC()

In [71]:
model.fit(x_train, y_train)

SVC()

In [72]:
model.score(x_train,y_train)

0.6924643584521385

In [73]:
model.score(x_test,y_test)

0.6910569105691057

In [75]:
scaler = MinMaxScaler()

In [76]:
x_train = scaler.fit_transform(x_train)

In [77]:
x_test = scaler.transform(x_test)

In [78]:
model2 =SVC()

In [79]:
model2.fit(x_train, y_train)

SVC()

In [80]:
model2.score(x_train,y_train)

0.8044806517311609

In [81]:
model2.score(x_test,y_test)

0.8373983739837398

### 3. pipeline

In [92]:
trans_func = FunctionTransformer(fill_null)

In [93]:
pipeline = make_pipeline(MinMaxScaler(), trans_func, SVC())